In [13]:
%%writefile goose.py

import os

os.environ['CUDA_VISIBLE_DEVICES'] = "-1"

import tensorflow as tf
import numpy as np
import base64
import json
from kaggle_environments.envs.hungry_geese.hungry_geese import Action
from kaggle_environments import make

NUM_GRID = (7, 11)
NUM_CHANNEL = 6
NUM_ACT = 4

STOCK_X = tf.convert_to_tensor(np.zeros((*NUM_GRID, NUM_CHANNEL)), dtype='float32')
STOCK_ACT = [Action(i + 1) for i in range(NUM_ACT)]

class Block(tf.keras.layers.Layer):
    def __init__(self, flt, **kwargs):
        super(Block, self).__init__(**kwargs)
        
        self.dense = tf.keras.layers.Dense(flt, use_bias=False)
        self.bn = tf.keras.layers.BatchNormalization()
        
    def call(self, inp, training=False):
        x = self.dense(inp)
        x = self.bn(x, training)
        x = tf.nn.relu(x)
        
        return x

class Net(tf.keras.Model):
    def __init__(self, layers, out, stock):
        super(Net, self).__init__()

        self.stats = { 'layers': layers, 'out': out, 'stock': stock }
        self.stock = stock

        self.tower = []

        for l in layers:
            self.tower.append(Block(l))
        
        self.flt = tf.keras.layers.Flatten()
        self.out = tf.keras.layers.Dense(out)
    
    def call(self, inp, training=False):
        x = inp

        if len(x.shape) < 4:
            x = tf.expand_dims(x, 0)
            
        x = self.flt(x)

        for block in self.tower:
            x = block(x, training=training)
            
        x = self.out(x)
            
        return x
    
def act_to_id(act):
    if act.name == 'NORTH':
        return 0
    elif act.name == 'EAST':
        return 1
    elif act.name == 'SOUTH':
        return 2
    elif act.name == 'WEST':
        return 3

def pos_to_coord(pos):
    return pos // NUM_GRID[1], pos % NUM_GRID[1]

def obs_to_x(obs, acts):
    x = [[[0 for _ in range(NUM_CHANNEL)] for _ in range(NUM_GRID[1])] for _ in range(NUM_GRID[0])]
    index = obs['index']
    foods = obs['food']
    geese = obs['geese']

    rc, cc = pos_to_coord(geese[index][0])
    rc = round((NUM_GRID[0] - 1) / 2) - rc
    cc = round((NUM_GRID[1] - 1) / 2) - cc
    
    for food in foods:
        r, c = pos_to_coord(food)
        
        r = (r + rc) % NUM_GRID[0]
        c = (c + cc) % NUM_GRID[1]

        x[r][c][5] = 1

    for i, goose in enumerate(geese):
        if goose:
            r, c = pos_to_coord(goose[0])
            
            r = (r + rc) % NUM_GRID[0]
            c = (c + cc) % NUM_GRID[1]
            
            x[r][c][4] = 1
            
            if acts[i] != None:
                x[r][c][act_to_id(acts[i])] = -1
            
            for ii, block in enumerate(goose):
                if ii == 0:
                    continue
                
                r, c = pos_to_coord(block)
                rt, ct = pos_to_coord(goose[ii - 1])
                
                d = None
                
                if (r - 1) % NUM_GRID[0] == rt:
                    d = 0
                elif (c + 1) % NUM_GRID[1] == ct:
                    d = 1
                elif (r + 1) % NUM_GRID[0] == rt:
                    d = 2
                elif (c - 1) % NUM_GRID[1] == ct:
                    d = 3
                
                r = (r + rc) % NUM_GRID[0]
                c = (c + cc) % NUM_GRID[1]

                x[r][c][4] = 1
                x[r][c][d] = 1
    
    return tf.convert_to_tensor(x, dtype='float32')

shapes = [[462, 512], [512], [512], [512, 128], [128], [128], [128, 128], [128], [128], [128, 256], [256], [256], [512], [512], [128], [128], [128], [128], [256], [256], [256, 4], [4]]

weights = ["...", "vjvfO7s7tDo="]

weights = [np.frombuffer(base64.decodebytes(s.encode('utf-8')), dtype=np.half).reshape(shapes[i]) for i, s in enumerate(weights)]
weights = [arr.astype(np.float32) for arr in weights]

critic = Net([512, 128, 128, 256], NUM_ACT, STOCK_X)
critic(critic.stock)
critic.set_weights(weights)

heads = None

def agent(obs, _):
    global critic, heads
    
    acts = [None] * len(obs['geese'])
        
    if heads != None:
        for i, head in enumerate(heads):
            if head and obs['geese'][i]:
                r, c = pos_to_coord(head)
                rr, cc = pos_to_coord(obs['geese'][i][0])

                if (r - 1) % NUM_GRID[0] == rr:
                    acts[i] = STOCK_ACT[0]
                elif (c + 1) % NUM_GRID[1] == cc:
                    acts[i] = STOCK_ACT[1]
                elif (r + 1) % NUM_GRID[0] == rr:
                    acts[i] = STOCK_ACT[2]
                elif (c - 1) % NUM_GRID[1] == cc:
                    acts[i] = STOCK_ACT[3]

    heads = list()

    for goose in obs['geese']:
        if goose:
            heads.append(goose[0])
        else:
            heads.append(None)
            
    x = obs_to_x(obs, acts)
    
    scores = [float(score) for score in critic(x)[0]]
    
    i = scores.index(max(scores))
    
    return STOCK_ACT[i].name

Overwriting goose.py


In [14]:
%%writefile new.py

import os

os.environ['CUDA_VISIBLE_DEVICES'] = "-1"

import tensorflow as tf
import numpy as np
import base64
import json
from kaggle_environments.envs.hungry_geese.hungry_geese import Action
from kaggle_environments import make

NUM_GRID = (7, 11)
NUM_CHANNEL = 7
NUM_ACT = 4

STOCK_X = tf.zeros((*NUM_GRID, NUM_CHANNEL), dtype='int8')
STOCK_ACT = [Action(i + 1) for i in range(NUM_ACT)]

class Block(tf.keras.layers.Layer):
    def __init__(self, flt, **kwargs):
        super(Block, self).__init__(**kwargs)
        
        self.dense = tf.keras.layers.Dense(flt, use_bias=False, dtype='float16')
        self.bn = tf.keras.layers.BatchNormalization()
        
    def call(self, inp, training=False):
        x = self.dense(inp)
        x = self.bn(x, training)
        x = tf.nn.relu(x)
        
        return x

class Net(tf.keras.Model):
    def __init__(self, layers, out, stock):
        super(Net, self).__init__()

        self.stats = { 'layers': layers, 'out': out, 'stock': stock }
        self.stock = stock

        self.tower = []

        for l in layers:
            self.tower.append(Block(l))
        
        self.flt = tf.keras.layers.Flatten()
        self.out = tf.keras.layers.Dense(out, dtype='float16')
    
    def call(self, inp, training=False):
        x = inp

        if len(x.shape) < 4:
            x = tf.expand_dims(x, 0)
            
        x = self.flt(x)

        for block in self.tower:
            x = block(x, training=training)
            
        x = self.out(x)
            
        return x
    
def act_to_id(act):
    if act.name == 'NORTH':
        return 0
    elif act.name == 'EAST':
        return 1
    elif act.name == 'SOUTH':
        return 2
    elif act.name == 'WEST':
        return 3

def pos_to_coord(pos):
    return pos // NUM_GRID[1], pos % NUM_GRID[1]

def obs_to_x(obs, acts):
    x = [[[0 for _ in range(NUM_CHANNEL)] for _ in range(NUM_GRID[1])] for _ in range(NUM_GRID[0])]
    index = obs['index']
    foods = obs['food']
    geese = obs['geese']

    rc, cc = pos_to_coord(geese[index][0])
    rc = round((NUM_GRID[0] - 1) / 2) - rc
    cc = round((NUM_GRID[1] - 1) / 2) - cc
    
    for food in foods:
        r, c = pos_to_coord(food)
        
        r = (r + rc) % NUM_GRID[0]
        c = (c + cc) % NUM_GRID[1]

        x[r][c][6] = 1

    for i, goose in enumerate(geese):
        if goose:
            r, c = pos_to_coord(goose[0])
            
            r = (r + rc) % NUM_GRID[0]
            c = (c + cc) % NUM_GRID[1]
            
            x[r][c][4] = 1
            x[r][c][5] = 1
            
            if acts[i] != None:
                x[r][c][act_to_id(acts[i])] = 1
            
            for ii, block in enumerate(goose):
                if ii == 0:
                    continue
                
                r, c = pos_to_coord(block)
                rt, ct = pos_to_coord(goose[ii - 1])
                
                d = None
                
                if (r - 1) % NUM_GRID[0] == rt:
                    d = 0
                elif (c + 1) % NUM_GRID[1] == ct:
                    d = 1
                elif (r + 1) % NUM_GRID[0] == rt:
                    d = 2
                elif (c - 1) % NUM_GRID[1] == ct:
                    d = 3
                
                r = (r + rc) % NUM_GRID[0]
                c = (c + cc) % NUM_GRID[1]

                x[r][c][4] = 1
                x[r][c][d] = 1
    
    return tf.convert_to_tensor(x, dtype='int8')

shapes = json.loads('[[[539, 256], true], [[256], false], [[256], false], [[256, 256], true], [[256], false], [[256], false], [[256, 256], true], [[256], false], [[256], false], [[256, 256], true], [[256], false], [[256], false], [[256], false], [[256], false], [[256], false], [[256], false], [[256], false], [[256], false], [[256], false], [[256], false], [[256, 4], true], [[4], true]]')

weights = ["...", "Kz4jPb1AUkA="]

weights = [np.frombuffer(base64.decodebytes(s.encode('ascii')), dtype=(np.float16 if shapes[i][1] else np.float32)).reshape(shapes[i][0]) for i, s in enumerate(weights)]

critic = Net([256, 256, 256, 256], NUM_ACT, STOCK_X)
critic(critic.stock)
critic.set_weights(weights)

heads = None

def agent(obs, _):
    global critic, heads
    
    acts = [None] * len(obs['geese'])
        
    if heads != None:
        for i, head in enumerate(heads):
            if head and obs['geese'][i]:
                r, c = pos_to_coord(head)
                rr, cc = pos_to_coord(obs['geese'][i][0])

                if (r - 1) % NUM_GRID[0] == rr:
                    acts[i] = STOCK_ACT[0]
                elif (c + 1) % NUM_GRID[1] == cc:
                    acts[i] = STOCK_ACT[1]
                elif (r + 1) % NUM_GRID[0] == rr:
                    acts[i] = STOCK_ACT[2]
                elif (c - 1) % NUM_GRID[1] == cc:
                    acts[i] = STOCK_ACT[3]

    heads = list()

    for goose in obs['geese']:
        if goose:
            heads.append(goose[0])
        else:
            heads.append(None)
            
    x = obs_to_x(obs, acts)
    
    scores = [float(score) for score in critic(x)[0]]
    
    i = scores.index(max(scores))
    
    print(obs['index'], scores)
    
    return STOCK_ACT[i].name

Overwriting new.py


In [15]:
%%writefile other.py
import pickle
import bz2
import base64
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Conv2D, Activation, Lambda, Add, BatchNormalization, Input
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l1_l2, l2


# Neural Network for Hungry Geese
def TorusConv2D(x, ch, kernel, padding="same", strides=1, weight_decay=2e-3):
    x = Lambda(lambda x: K.tile(x, n=(1,3,3,1)), 
               output_shape=lambda input_shape: (None, 3*input_shape[1], 3*input_shape[2], input_shape[3]))(x)
    
    x = Conv2D(ch, kernel, padding=padding, strides=strides,
                      kernel_regularizer=l2(weight_decay))(x)
    
    x = Lambda(lambda x: x[:,int(x.shape[1]/3):2*int(x.shape[1]/3), int(x.shape[2]/3):2*int(x.shape[2]/3),:], 
               output_shape=lambda input_shape: (None, int(input_shape[1]/3), int(input_shape[2]/3), input_shape[3]))(x)
    return x

def conv_bn_relu(x0, ch, kernel, padding="same", strides=1, weight_decay=2e-3, add=False):
    x = TorusConv2D(x0, ch, kernel, padding=padding, strides=strides,
                      weight_decay=weight_decay)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    if add:
        x = Add()([x0, x])
    return x

def GeeseNet(input_shape=(7, 11, 17), layers=12, filters=32, weight_decay=2e-3):
    input = Input(input_shape)
    x = conv_bn_relu(input, filters, 3)
    
    for i in range(layers):
        x = conv_bn_relu(x, filters, 3, add=True)
    
    x = GlobalAveragePooling2D()(x)
    
    output = Dense(4, activation='softmax', kernel_regularizer=l1_l2(l1=0.0005, l2=0.0005))(x)   
    model = Model(input, output)
    #model.compile(optimizer=RadaBelief(learning_rate=1e-3, epsilon=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])    
    
    return model

# Input for Neural Network
def centerize(b):
    dy, dx = np.where(b[0])
    centerize_y = (np.arange(0,7)-3+dy[0])%7
    centerize_x = (np.arange(0,11)-5+dx[0])%11
    
    b = b[:, centerize_y,:]
    b = b[:, :,centerize_x]
    
    return b

def make_input(obses):
    b = np.zeros((17, 7 * 11), dtype=np.float32)
    obs = obses[-1]

    for p, pos_list in enumerate(obs['geese']):
        # head position
        for pos in pos_list[:1]:
            b[0 + (p - obs['index']) % 4, pos] = 1
        # tip position
        for pos in pos_list[-1:]:
            b[4 + (p - obs['index']) % 4, pos] = 1
        # whole position
        for pos in pos_list:
            b[8 + (p - obs['index']) % 4, pos] = 1
            
    # previous head position
    if len(obses) > 1:
        obs_prev = obses[-2]
        for p, pos_list in enumerate(obs_prev['geese']):
            for pos in pos_list[:1]:
                b[12 + (p - obs['index']) % 4, pos] = 1

    # food
    for pos in obs['food']:
        b[16, pos] = 1
        
    b = b.reshape(-1, 7, 11)
    b = centerize(b)

    return b


# Load Keras Model
weight = b'...'

model = GeeseNet(input_shape=(7, 11, 17), layers=12, filters=32, weight_decay=1e-7)
model.set_weights(pickle.loads(bz2.decompress(base64.b64decode(weight))))

# Main Function of Agent

obses = []

def agent(obs_dict, config_dict):
    obses.append(obs_dict)

    X_test = make_input(obses)
    X_test = np.transpose(X_test, (1,2,0))
    X_test = X_test.reshape(-1,7,11,17) # channel last.
    
    # avoid suicide
    obstacles = X_test[:,:,:,[8,9,10,11,12]].max(axis=3) - X_test[:,:,:,[4,5,6,7]].max(axis=3) # body + opposite_side - my tail
    obstacles = np.array([obstacles[0,2,5], obstacles[0,4,5], obstacles[0,3,4], obstacles[0,3,6]])
    
    y_pred = model.predict(X_test) - obstacles

    
    actions = ['NORTH', 'SOUTH', 'WEST', 'EAST']
    return actions[np.argmax(y_pred)]

Overwriting other.py


In [16]:
from kaggle_environments import make

env = make('hungry_geese', debug=True)
foo = env.run(['new.py', 'new.py', 'new.py', 'new.py'])
print(foo[0])
env.render(mode='ipython')

0 [12.4296875, 11.703125, 12.953125, 13.703125]
1 [12.6171875, 13.96875, 14.8828125, 11.359375]
2 [15.7890625, 15.2578125, 11.9921875, 11.953125]
3 [15.625, 12.15625, 12.109375, 15.953125]
0 [12.84375, -29.140625, 14.5625, 14.03125]
1 [-61.15625, 13.859375, 14.1015625, 12.40625]
2 [15.59375, 16.078125, -21.28125, 11.828125]
3 [18.609375, 2.3515625, 13.7265625, 18.328125]
0 [-63.46875, 11.6640625, 15.71875, 16.171875]
1 [-44.0, 16.859375, 16.96875, 13.046875]
2 [17.96875, 19.125, 13.8359375, -42.09375]
3 [15.7890625, 13.5546875, 4.609375, 20.359375]
0 [13.34375, 0.2998046875, 17.71875, 14.171875]
1 [-53.40625, -42.59375, 12.828125, 16.3125]
2 [21.859375, 16.203125, 13.5234375, -73.5625]
3 [21.140625, -140.375, 20.171875, 19.640625]
0 [-2.76171875, 12.109375, -63.0625, 15.265625]
1 [14.1015625, -11.265625, 13.84375, 12.640625]
2 [19.921875, 21.625, -78.9375, 26.640625]
3 [22.96875, 22.03125, -49.25, 21.25]
0 [13.5078125, -81.4375, -138.125, 12.515625]
1 [20.375, 15.15625, -3.220703125, 1